<a href="https://colab.research.google.com/github/kumuds4/BCH/blob/master/Making_the_Most_of_your_Colab_Subscription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making the Most of your Colab Subscription



## Faster GPUs

Users who have purchased one of Colab's paid plans have access to faster GPUs and more memory. You can upgrade your notebook's GPU settings in `Runtime > Change runtime type` in the menu to select from several accelerator options, subject to availability.

The free of charge version of Colab grants access to Nvidia's T4 GPUs subject to quota restrictions and availability.

You can see what GPU you've been assigned at any time by executing the following cell. If the execution result of running the code cell below is "Not connected to a GPU", you can change the runtime by going to `Runtime > Change runtime type` in the menu to enable a GPU accelerator, and then re-execute the code cell.


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In order to use a GPU with your notebook, select the `Runtime > Change runtime type` menu, and then set the hardware accelerator to the desired option.

## More memory

Users who have purchased one of Colab's paid plans have access to high-memory VMs when they are available. More powerful GPUs are always offered with high-memory VMs.



You can see how much memory you have available at any time by running the following code cell. If the execution result of running the code cell below is "Not using a high-RAM runtime", then you can enable a high-RAM runtime via `Runtime > Change runtime type` in the menu. Then select High-RAM in the Runtime shape toggle button. After, re-execute the code cell.


In [ ]:
import psutil

ram_gb = psutil.virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

## Longer runtimes

All Colab runtimes are reset after some period of time (which is faster if the runtime isn't executing code). Colab Pro and Pro+ users have access to longer runtimes than those who use Colab free of charge.

## Background execution

Colab Pro+ users have access to background execution, where notebooks will continue executing even after you've closed a browser tab. This is always enabled in Pro+ runtimes as long as you have compute units available.



## Relaxing resource limits in Colab Pro

Your resources are not unlimited in Colab. To make the most of Colab, avoid using resources when you don't need them. For example, only use a GPU when required and close Colab tabs when finished.



If you encounter limitations, you can relax those limitations by purchasing more compute units via Pay As You Go. Anyone can purchase compute units via [Pay As You Go](https://colab.research.google.com/signup); no subscription is required.

## Send us feedback!

If you have any feedback for us, please let us know. The best way to send feedback is by using the Help > 'Send feedback...' menu. If you encounter usage limits in Colab Pro consider subscribing to Pro+.

If you encounter errors or other issues with billing (payments) for Colab Pro, Pro+, or Pay As You Go, please email [colab-billing@google.com](mailto:colab-billing@google.com).

## More Resources

### Working with Notebooks in Colab
- [Overview of Colab](/notebooks/basic_features_overview.ipynb)
- [Guide to Markdown](/notebooks/markdown_guide.ipynb)
- [Importing libraries and installing dependencies](/notebooks/snippets/importing_libraries.ipynb)
- [Saving and loading notebooks in GitHub](https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb)
- [Interactive forms](/notebooks/forms.ipynb)
- [Interactive widgets](/notebooks/widgets.ipynb)

<a name="working-with-data"></a>
### Working with Data
- [Loading data: Drive, Sheets, and Google Cloud Storage](/notebooks/io.ipynb)
- [Charts: visualizing data](/notebooks/charts.ipynb)
- [Getting started with BigQuery](/notebooks/bigquery.ipynb)

### Machine Learning Crash Course
These are a few of the notebooks from Google's online Machine Learning course. See the [full course website](https://developers.google.com/machine-learning/crash-course/) for more.
- [Intro to Pandas DataFrame](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/pandas_dataframe_ultraquick_tutorial.ipynb)
- [Linear regression with tf.keras using synthetic data](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/linear_regression_with_synthetic_data.ipynb)


<a name="using-accelerated-hardware"></a>
### Using Accelerated Hardware
- [TensorFlow with GPUs](/notebooks/gpu.ipynb)
- [TPUs in Colab](/notebooks/tpu.ipynb)

<a name="machine-learning-examples"></a>

## Machine Learning Examples

To see end-to-end examples of the interactive machine learning analyses that Colab makes possible, check out these tutorials using models from [TensorFlow Hub](https://tfhub.dev).

A few featured examples:

- [Retraining an Image Classifier](https://tensorflow.org/hub/tutorials/tf2_image_retraining): Build a Keras model on top of a pre-trained image classifier to distinguish flowers.
- [Text Classification](https://tensorflow.org/hub/tutorials/tf2_text_classification): Classify IMDB movie reviews as either *positive* or *negative*.
- [Style Transfer](https://tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization): Use deep learning to transfer style between images.
- [Multilingual Universal Sentence Encoder Q&A](https://tensorflow.org/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa): Use a machine learning model to answer questions from the SQuAD dataset.
- [Video Interpolation](https://tensorflow.org/hub/tutorials/tweening_conv3d): Predict what happened in a video between the first and the last frame.


In [19]:
# Comprehensive Polar Code Simulation Framework
#Today May 17, 2025
# Essential Scientific and Deep Learning Libraries
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Machine Learning and Data Handling
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Visualization and Scientific Computing
import matplotlib.pyplot as plt
import scipy.special as sps

# System and Utilities
import logging
import traceback
import sys

# Logging Configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s]: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Device Configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using Device: {DEVICE}")

#Part 2: Polar Code Generator with CRC


class PolarCodeGenerator:
    def __init__(self, N, K, crc_type='CRC-7'):
        """
        Polar Code Generator with CRC support

        Args:
            N (int): Total code length
            K (int): Information bit length
            crc_type (str): CRC polynomial type
        """
        self.N = N
        self.K = K
        self.crc_type = crc_type

        # CRC Polynomials
        self.crc_polynomials = {
            'CRC-7': {
                'polynomial': [1, 1, 1, 0, 0, 1, 1],
                'length': 7
            }
        }

    def generate_info_bits(self):
        """
        Generate random information bits

        Returns:
            np.ndarray: Random information bits
        """
        return np.random.randint(2, size=self.K)

    def compute_crc(self, bits):
        """
        Compute CRC checksum using polynomial division

        Args:
            bits (np.ndarray): Input bits

        Returns:
            np.ndarray: CRC checksum bits
        """
        poly_info = self.crc_polynomials.get(self.crc_type)
        if not poly_info:
            raise ValueError(f"Unsupported CRC type: {self.crc_type}")

        polynomial = poly_info['polynomial']
        crc_length = poly_info['length']

        # Convert input to list and pad
        message = bits.tolist() + [0] * crc_length

        # Polynomial long division
        for i in range(len(message) - crc_length):
            if message[i] == 1:
                for j in range(crc_length + 1):
                    message[i + j] ^= polynomial[j] if j < len(polynomial) else 0

        # Return the last 'crc_length' bits
        return np.array(message[-crc_length:], dtype=int)

    def polar_encode(self, info_bits):
        """
        Polar Code Encoding with CRC

        Args:
            info_bits (np.ndarray): Information bits

        Returns:
            np.ndarray: Encoded codeword
        """
        # Append CRC
        crc_bits = self.compute_crc(info_bits)
        extended_info_bits = np.concatenate([info_bits, crc_bits])

        # Basic polar encoding (placeholder)
        codeword = np.zeros(self.N, dtype=int)
        codeword[:len(extended_info_bits)] = extended_info_bits

        return codeword

    def verify_codeword(self, codeword):
        """
        Verify codeword using CRC

        Args:
            codeword (np.ndarray): Received codeword

        Returns:
            bool: True if CRC check passes, False otherwise
        """
        poly_info = self.crc_polynomials.get(self.crc_type)
        if not poly_info:
            raise ValueError(f"Unsupported CRC type: {self.crc_type}")

        crc_length = poly_info['length']

        # Extract information and CRC bits
        info_bits = codeword[:-crc_length]
        received_crc = codeword[-crc_length:]

        # Compute CRC of information bits
        computed_crc = self.compute_crc(info_bits)

        # Compare received and computed CRC
        return np.array_equal(received_crc, computed_crc)

  #Part 3
class EnhancedChannelSimulator:
    def __init__(self, channel_type='AWGN'):
        """
        Advanced Channel Simulator for communication systems

        Args:
            channel_type (str): Channel type ('AWGN' or 'Rayleigh')
        """
        self.channel_type = channel_type
        logging.info(f"Initializing {channel_type} Channel Simulator")

    def simulate(self, encoded_signal, snr_db):
        """
        Simulate signal transmission through specified channel

        Args:
            encoded_signal (np.ndarray): Input encoded signal
            snr_db (float): Signal-to-Noise Ratio in decibels

        Returns:
            np.ndarray: Received signal after channel effects
        """
        try:
            # Convert input to numpy array
            encoded_signal = np.array(encoded_signal, dtype=float)

            # Convert bits {0,1} to BPSK: {+1, -1}
            bpsk_signal = 1 - 2 * encoded_signal

            # Convert SNR from dB to linear scale
            snr_linear = 10 ** (snr_db / 10)

            # Compute signal power
            signal_power = np.mean(bpsk_signal**2)

            # Noise power calculation
            noise_power = signal_power / snr_linear
            noise_std = np.sqrt(noise_power / 2.0)

            # Channel-specific simulation
            if self.channel_type == 'AWGN':
                # Additive White Gaussian Noise
                noise = np.random.normal(0, noise_std, bpsk_signal.shape)
                received_signal = bpsk_signal + noise

            elif self.channel_type == 'Rayleigh':
                # Rayleigh Fading Channel
                fading = np.random.rayleigh(scale=1.0, size=bpsk_signal.shape)
                noise = np.random.normal(0, noise_std, bpsk_signal.shape)
                received_signal = fading * bpsk_signal + noise

            else:
                raise ValueError(f"Unsupported channel type: {self.channel_type}")

            # Convert back to binary representation
            return (received_signal > 0).astype(float)

        except Exception as e:
            logging.error(f"Channel simulation error: {e}")
            return encoded_signal

    def compute_theoretical_performance(self, block_length, snr_linear):
        """
        Compute theoretical Bit Error Probability (BEP) and Block Error Probability (BLER)

        Args:
            block_length (int): Length of the code block
            snr_linear (np.ndarray): SNR in linear scale

        Returns:
            tuple: Theoretical BEP and BLER
        """
        try:
            if self.channel_type == 'AWGN':
                # AWGN Channel Theoretical Performance
                # Bit Error Probability using Q-function
                bep = 0.5 * sps.erfc(np.sqrt(snr_linear))

            elif self.channel_type == 'Rayleigh':
                # Rayleigh Fading Channel Theoretical Performance
                # Average Bit Error Probability for Rayleigh fading
                bep = 0.5 * (1 - np.sqrt(snr_linear / (1 + snr_linear)))

            else:
                raise ValueError(f"Unsupported channel type: {self.channel_type}")

            # Block Error Probability (assuming independent bit errors)
            bler = 1 - (1 - bep) ** block_length

            return bep, bler

        except Exception as e:
            logging.error(f"Theoretical performance computation error: {e}")
            # Return default values if computation fails
            return np.zeros_like(snr_linear), np.ones_like(snr_linear)

    def plot_channel_capacity(self, snr_range):
        """
        Plot channel capacity for the specific channel type

        Args:
            snr_range (np.ndarray): Range of SNR values in dB
        """
        plt.figure(figsize=(10, 6))

        # Compute channel capacities
        snr_linear = 10 ** (snr_range / 10)
        capacities = [np.log2(1 + snr) for snr in snr_linear]

        plt.plot(snr_range, capacities, label=f'{self.channel_type} Channel')
        plt.title(f'Channel Capacity - {self.channel_type} Channel')
        plt.xlabel('SNR (dB)')
        plt.ylabel('Capacity (bits/channel use)')
        plt.grid(True)
        plt.legend()
        plt.show()

# Utility function for dataset preparation
################################################
#Latest dataset prep
def prepare_polar_dataset(polar_code_gen, num_samples, snr_db=5, channel_type="AWGN"):
    """
    Prepare dataset for Polar Code simulation with robust preprocessing

    Args:
        polar_code_gen (PolarCodeGenerator): Polar code generator
        num_samples (int): Number of samples to generate
        snr_db (float): Signal-to-Noise Ratio in dB
        channel_type (str): Channel type

    Returns:
        tuple: Input features and corresponding labels
    """
    # Create channel simulator
    channel_simulator = EnhancedChannelSimulator(channel_type=channel_type)

    # Initialize storage
    X = []
    y = []

    for _ in range(num_samples):
        # Generate information bits
        info_bits = polar_code_gen.generate_info_bits()

        # Encode polar code
        encoded_signal = polar_code_gen.polar_encode(info_bits)

        # Channel simulation
        received_signal = channel_simulator.simulate(encoded_signal, snr_db)

        # Store features and labels
        X.append(received_signal)
        y.append(info_bits)  # Use original info_bits as target

    # Convert to numpy arrays
    X = np.array(X)
    y = np.array(y)

    # Print dataset information
    print("\n📊 Dataset Preparation Diagnostics:")
    print(f"Input Tensor Shape (X): {X.shape}")
    print(f"Label Tensor Shape (y): {y.shape}")

    return X, y

class DecoderTrainer:
    def __init__(self, model, learning_rate=1e-3):
        """
        Decoder Trainer with enhanced tensor handling
        """
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = model.to(self.device)
        self.criterion = nn.BCELoss()
        self.optimizer = optim.Adam(
            self.model.parameters(),
            lr=learning_rate,
            weight_decay=1e-5
        )
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer,
            mode='min',
            factor=0.5,
            patience=5,
            verbose=True
        )
        self.train_losses = []
        self.val_losses = []

    def _preprocess_tensor(self, X, y):
        """
        Comprehensive tensor preprocessing
        """
        # Ensure X is a tensor
        if not isinstance(X, torch.Tensor):
            X = torch.FloatTensor(X)

        # Ensure y is a tensor
        if not isinstance(y, torch.Tensor):
            y = torch.FloatTensor(y)

        # Flatten multi-dimensional inputs
        X = X.view(X.size(0), -1)

        # Ensure y is 2D with correct shape
        y = y.view(-1, 1).float()

        # Print diagnostic information
        print("\n🔍 Tensor Preprocessing Diagnostics:")
        print(f"Processed X shape: {X.shape}")
        print(f"Processed y shape: {y.shape}")

        return X, y

    def train(self, X, y, epochs=50, batch_size=32, validation_split=0.2):
        """
        Enhanced training method with robust tensor handling
        """
        # Preprocess tensors
        X, y = self._preprocess_tensor(X, y)

        # Move to device
        X = X.to(self.device)
        y = y.to(self.device)

        # Split into train and validation
        train_size = int((1 - validation_split) * len(X))
        X_val, y_val = X[train_size:], y[train_size:]
        X_train, y_train = X[:train_size], y[:train_size]

        # Create data loaders
        train_dataset = TensorDataset(X_train, y_train)
        train_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            shuffle=True
        )

        val_dataset = TensorDataset(X_val, y_val)
        val_loader = DataLoader(
            val_dataset,
            batch_size=batch_size,
            shuffle=False
        )

        # Training loop
        for epoch in range(epochs):
            # Training phase
            self.model.train()
            train_loss = self._train_epoch(train_loader)
            self.train_losses.append(train_loss)

            # Validation phase
            self.model.eval()
            val_loss = self._validate(val_loader)
            self.val_losses.append(val_loss)

            # Learning rate scheduling
            self.scheduler.step(val_loss)

            # Print progress
            print(f"Epoch [{epoch+1}/{epochs}], "
                  f"Train Loss: {train_loss:.4f}, "
                  f"Val Loss: {val_loss:.4f}")

        return self.train_losses, self.val_losses

    def predict(self, X):
        """
        Enhanced prediction method with tensor preprocessing
        """
        # Ensure input is a tensor
        if not isinstance(X, torch.Tensor):
            X = torch.FloatTensor(X)

        # Flatten multi-dimensional inputs
        X = X.view(X.size(0), -1)

        # Move to device
        X = X.to(self.device)

        # Set model to evaluation mode
        self.model.eval()

        # Predict
        with torch.no_grad():
            outputs = self.model(X)

        return outputs.cpu().numpy().flatten()

class EnhancedRNNDecoder(nn.Module):
    def __init__(self, input_size):
        """
        Simplified RNN Decoder with robust input handling
        """
        super(EnhancedRNNDecoder, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        """
        Forward pass with flexible input handling
        """
        # Ensure input is 2D
        if x.dim() == 1:
            x = x.unsqueeze(0)

        # Flatten if multi-dimensional
        if x.dim() > 2:
            x = x.view(x.size(0), -1)

        return self.model(x).squeeze(-1)

###################################################


   #part 4



class DecoderTrainer:
    def __init__(self, model, learning_rate=1e-3):
        """
        Trainer for Neural Network Decoders

        Args:
            model (nn.Module): Neural network model
            learning_rate (float): Learning rate for optimization
        """
        # Device configuration
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Model setup
        self.model = model.to(self.device)

        # Loss and Optimization
        self.criterion = nn.BCELoss()
        self.optimizer = optim.Adam(
            self.model.parameters(),
            lr=learning_rate,
            weight_decay=1e-5
        )

        # Learning rate scheduler
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer,
            mode='min',
            factor=0.5,
            patience=5,
            verbose=True
        )

        # Training tracking
        self.train_losses = []
        self.val_losses = []

    def train(self, X, y, epochs=50, batch_size=32, validation_split=0.2):
        """
        Train the decoder model

        Args:
            X (torch.Tensor): Input features
            y (torch.Tensor): Target labels
            epochs (int): Number of training epochs
            batch_size (int): Batch size for training
            validation_split (float): Proportion of data for validation

        Returns:
            tuple: Training and validation losses
        """
        # Ensure tensors are on correct device and have correct shape
        X = X.to(self.device)
        y = y.to(self.device)

        # Reshape if needed
        if X.dim() == 3:
            X = X.view(X.size(0), -1)

        # Ensure y is 2D
        y = y.view(-1, 1).float()

        # Split into train and validation
        train_size = int((1 - validation_split) * len(X))
        X_val, y_val = X[train_size:], y[train_size:]
        X_train, y_train = X[:train_size], y[:train_size]

        # Create data loaders
        train_dataset = TensorDataset(X_train, y_train)
        train_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            shuffle=True
        )

        val_dataset = TensorDataset(X_val, y_val)
        val_loader = DataLoader(
            val_dataset,
            batch_size=batch_size,
            shuffle=False
        )

        # Training loop
        for epoch in range(epochs):
            # Training phase
            self.model.train()
            train_loss = self._train_epoch(train_loader)
            self.train_losses.append(train_loss)

            # Validation phase
            self.model.eval()
            val_loss = self._validate(val_loader)
            self.val_losses.append(val_loss)

            # Learning rate scheduling
            self.scheduler.step(val_loss)

            # Print progress
            logging.info(f"Epoch [{epoch+1}/{epochs}], "
                         f"Train Loss: {train_loss:.4f}, "
                         f"Val Loss: {val_loss:.4f}")

        return self.train_losses, self.val_losses

    def _train_epoch(self, dataloader):
        """
        Train for one epoch

        Args:
            dataloader (DataLoader): Training data loader

        Returns:
            float: Average training loss
        """
        total_loss = 0
        for batch_X, batch_y in dataloader:
            # Move to device
            batch_X = batch_X.to(self.device)
            batch_y = batch_y.to(self.device)

            # Zero gradients
            self.optimizer.zero_grad()

            # Forward pass
            outputs = self.model(batch_X)

            # Compute loss
            loss = self.criterion(outputs, batch_y)

            # Backward pass
            loss.backward()

            # Optimize
            self.optimizer.step()

            # Accumulate loss
            total_loss += loss.item()

        return total_loss / len(dataloader)

    def _validate(self, dataloader):
        """
        Validate model performance

        Args:
            dataloader (DataLoader): Validation data loader

        Returns:
            float: Validation loss
        """
        total_loss = 0
        with torch.no_grad():
            for batch_X, batch_y in dataloader:
                # Move to device
                batch_X = batch_X.to(self.device)
                batch_y = batch_y.to(self.device)

                # Forward pass
                outputs = self.model(batch_X)

                # Compute loss
                loss = self.criterion(outputs, batch_y)

                # Accumulate loss
                total_loss += loss.item()

        return total_loss / len(dataloader)

    def predict(self, X):
        """
        Make predictions using the trained model

        Args:
            X (np.ndarray or torch.Tensor): Input data

        Returns:
            np.ndarray: Predicted probabilities
        """
        # Ensure input is a tensor
        if not isinstance(X, torch.Tensor):
            X = torch.FloatTensor(X).to(self.device)
        else:
            X = X.to(self.device)

        # Reshape if needed
        if X.dim() == 3:
            X = X.view(X.size(0), -1)

        # Set model to evaluation mode
        self.model.eval()

        # Predict
        with torch.no_grad():
            outputs = self.model(X)

        return outputs.cpu().numpy().flatten()

#Part 5
def performance_comparison(decoder, channel_simulator, polar_code_gen, snr_range, channel_type, list_sizes=[1, 4, 8]):
    """
    Comprehensive performance comparison across different list sizes

    Args:
        decoder (DecoderTrainer): Trained decoder
        channel_simulator (EnhancedChannelSimulator): Channel simulator
        polar_code_gen (PolarCodeGenerator): Polar code generator
        snr_range (np.ndarray): SNR range for evaluation
        channel_type (str): Channel type
        list_sizes (list): List sizes for decoding

    Returns:
        dict: Comprehensive performance results
    """
    # Convert SNR to linear scale
    snr_linear = 10 ** (snr_range / 10)

    # Theoretical Performance
    block_length = polar_code_gen.N
    theoretical_bep, theoretical_bler = channel_simulator.compute_theoretical_performance(
        block_length, snr_linear
    )

    # Results storage
    results = {
        'theoretical': {
            'bep': theoretical_bep,
            'bler': theoretical_bler
        },
        'simulated': {}
    }

    # Performance computation for each list size
    for list_size in list_sizes:
        # Initialize storage for this list size
        results['simulated'][list_size] = {
            'ber': [],
            'bler': []
        }

        # Compute performance for each SNR point
        for snr in snr_range:
            total_bit_errors = 0
            total_block_errors = 0
            total_trials = 1000

            for _ in range(total_trials):
                # Generate information bits
                info_bits = polar_code_gen.generate_info_bits()

                # Encode polar code
                encoded_signal = polar_code_gen.polar_encode(info_bits)

                # Channel simulation
                received_signal = channel_simulator.simulate(encoded_signal, snr)

                # Convert to tensor for decoding
                received_tensor = torch.FloatTensor(received_signal).reshape(1, -1)

                # Decode with list size simulation
                decoded_output = decoder.predict(received_tensor)

                # Adjust decoding based on list size
                list_threshold = 1 - (list_size / 10)
                decoded_bits = (decoded_output > list_threshold).flatten()

                # Compute errors
                bit_errors = np.sum(np.abs(info_bits - decoded_bits[:len(info_bits)]))
                block_error = bit_errors > 0

                total_bit_errors += bit_errors
                total_block_errors += block_error

            # Compute average errors
            avg_ber = total_bit_errors / (total_trials * len(info_bits))
            avg_bler = total_block_errors / total_trials

            # Store results
            results['simulated'][list_size]['ber'].append(avg_ber)
            results['simulated'][list_size]['bler'].append(avg_bler)

    return results

def plot_comprehensive_analysis(rnn_trainer, X_test, y_test, channel_name, train_losses, val_losses, performance_results, snr_range, list_sizes):
    """
    Create comprehensive performance visualization

    Args:
        rnn_trainer (DecoderTrainer): Trained decoder
        X_test (torch.Tensor): Test input data
        y_test (torch.Tensor): Test labels
        channel_name (str): Channel type
        train_losses (list): Training losses
        val_losses (list): Validation losses
        performance_results (dict): Performance computation results
        snr_range (np.ndarray): SNR range
        list_sizes (list): List sizes
    """
    plt.figure(figsize=(20, 15))
    plt.suptitle(f'Comprehensive Analysis - {channel_name} Channel', fontsize=16)

    # 1. Training and Validation Losses
    plt.subplot(2, 2, 1)
    plt.plot(train_losses, label='Training Loss', color='blue')
    plt.plot(val_losses, label='Validation Loss', color='red')
    plt.title('Training and Validation Losses')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # 2. Confusion Matrix
    plt.subplot(2, 2, 2)
    predictions = rnn_trainer.predict(X_test)
    pred_classes = (predictions > 0.5).astype(int).flatten()
    true_classes = y_test.numpy().flatten()

    cm = confusion_matrix(true_classes, pred_classes)
    im = plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar(im)

    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')

    # 3. BER Performance
    plt.subplot(2, 2, 3)
    plt.title(f'Bit Error Rate (BER) - {channel_name} Channel')
    plt.xlabel('SNR (dB)')
    plt.ylabel('BER (Log Scale)')

    # Theoretical BER
    plt.semilogy(snr_range, performance_results['theoretical']['bep'],
                 label='Theoretical BER', color='black', linestyle='--')

    # Simulated BER for different list sizes
    colors = ['red', 'green', 'blue']
    for idx, list_size in enumerate(list_sizes):
        plt.semilogy(snr_range, performance_results['simulated'][list_size]['ber'],
                     label=f'List Size {list_size}', color=colors[idx], marker='o')

    plt.legend()
    plt.grid(True, which='both', ls=':', alpha=0.7)

    # 4. BLER Performance
    plt.subplot(2, 2, 4)
    plt.title(f'Block Error Rate (BLER) - {channel_name} Channel')
    plt.xlabel('SNR (dB)')
    plt.ylabel('BLER (Log Scale)')

    # Theoretical BLER
    plt.semilogy(snr_range, performance_results['theoretical']['bler'],
                 label='Theoretical BLER', color='black', linestyle='--')

    # Simulated BLER for different list sizes
    for idx, list_size in enumerate(list_sizes):
        plt.semilogy(snr_range, performance_results['simulated'][list_size]['bler'],
                     label=f'List Size {list_size}', color=colors[idx], marker='o')

    plt.legend()
    plt.grid(True, which='both', ls=':', alpha=0.7)

    plt.tight_layout()
    plt.show()


#Part 6: Main Simulation Framework
########################
#Latest main()


##########################
def main():
    """
    Main simulation framework for Polar Code performance analysis
    """
    try:
        # Simulation Configuration
        BLOCK_LENGTH = 32
        INFO_BITS = 16
        LEARNING_RATE = 1e-3
        EPOCHS = 50
        BATCH_SIZE = 32
        NUM_SAMPLES = 5000

        # SNR Ranges
        SNR_RANGE_AWGN = np.linspace(0, 5, 10)
        SNR_RANGE_RAYLEIGH = np.linspace(0, 10, 10)

        # List Sizes
        LIST_SIZES = [1, 4, 8]

        # Polar Code Generator
        polar_code_gen = PolarCodeGenerator(N=BLOCK_LENGTH, K=INFO_BITS)

        # Results storage
        results = {}

        # Channel Types
        channels = {
            'AWGN': EnhancedChannelSimulator(channel_type='AWGN'),
            'Rayleigh': EnhancedChannelSimulator(channel_type='Rayleigh')
        }

        # Iterate through channel types
        for channel_name, channel in channels.items():
            logging.info(f"Analyzing {channel_name} Channel")

            # Prepare Dataset
            X, y = prepare_polar_dataset(
                polar_code_gen,
                num_samples=NUM_SAMPLES,
                channel_type=channel_name
            )
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

            # Convert to PyTorch tensors
            X_train = torch.FloatTensor(X_train)
            X_test = torch.FloatTensor(X_test)
            y_train = torch.FloatTensor(y_train).view(-1, 1)
            y_test = torch.FloatTensor(y_test).view(-1, 1)

            # Enhanced RNN Decoder
            rnn_model = EnhancedRNNDecoder(input_size=X_train.size(1))
            rnn_trainer = DecoderTrainer(rnn_model)

            # Train Decoder
            train_losses, val_losses = rnn_trainer.train(
                X_train,
                y_train,
                epochs=EPOCHS,
                batch_size=BATCH_SIZE
            )

            # Determine SNR range
            snr_range = SNR_RANGE_AWGN if channel_name == 'AWGN' else SNR_RANGE_RAYLEIGH

            # Performance Comparison
            performance_results = performance_comparison(
                rnn_trainer,
                channel,
                polar_code_gen,
                snr_range,
                channel_name,
                LIST_SIZES
            )

            # Comprehensive Analysis Plot
            plot_comprehensive_analysis(
                rnn_trainer,
                X_test,
                y_test,
                channel_name,
                train_losses,
                val_losses,
                performance_results,
                snr_range,
                LIST_SIZES
            )

            # Store results
            results[channel_name] = {
                'decoder': rnn_trainer,
                'train_losses': train_losses,
                'val_losses': val_losses,
                'performance': performance_results
            }

        logging.info("🎉 Simulation Complete!")
        return results

    except Exception as e:
        logging.error(f"🆘 Comprehensive Simulation Error: {e}")
        traceback.print_exc()
        return None

# Execution
if __name__ == "__main__":
    main()
############################################





🚀 Using Device: cpu

📊 Dataset Preparation Diagnostics:
Input Tensor Shape (X): (5000, 32)
Label Tensor Shape (y): (5000, 16)


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
ERROR:root:🆘 Comprehensive Simulation Error: Size mismatch between tensors
Traceback (most recent call last):
  File "<ipython-input-19-137dff128298>", line 902, in main
    train_losses, val_losses = rnn_trainer.train(
                               ^^^^^^^^^^^^^^^^^^
  File "<ipython-input-19-137dff128298>", line 544, in train
    val_dataset = TensorDataset(X_val, y_val)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataset.py", line 205, in __init__
    assert all(
           ^^^^
AssertionError: Size mismatch between tensors
